In [ ]:
library(ggplot2)
library(tidyverse)
library(bigrquery)
library(lubridate)
library(table1)
library(margins)

In [ ]:
### prep census data ###

# collect census data from persons that also appear in primary care dataset - 2100773
census_sql_query <- bq_dataset_query(
    query = "SELECT person_id, AcademicYear, NCyearActual, AgeAtStartOfAcademicYear
             FROM `yhcr-prd-phm-bia-core.CB_FDM_DepartmentForEducation.src_census` census
             WHERE EXISTS(
                 SELECT person_id 
                 FROM `yhcr-prd-phm-bia-core.CB_FDM_PrimaryCare_v4.person` pc
                 WHERE census.person_id = pc.person_id
             )
             AND CAST(SUBSTR(AcademicYear, 1,4) AS INT64) > 2004 AND  
             CAST(SUBSTR(AcademicYear, 1,4) AS INT64) < 2019",
    x = "yhcr-prd-phm-bia-core.CB_FDM_DepartmentForEducation"
)

main_year_groups_factor <- factor(
    c("R", "1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11"), 
    levels = c("R", "1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11"),
    ordered = TRUE) 

census_data <- bq_table_download(census_sql_query) %>% 

    # aggregate entries to only one per person/academic year
    group_by(person_id, AcademicYear) %>% 
    summarise(age_ac_yr_start = first(AgeAtStartOfAcademicYear),
              year_group = first(NCyearActual)) %>%
    ungroup() %>%

    # where missing, infer academic year from age at start of year
    mutate(year_group = case_when( 
        !is.na(year_group) ~ year_group, 
        age_ac_yr_start == 4 ~ "R", 
        between(age_ac_yr_start, 5, 15) ~ 
                as.character(age_ac_yr_start - 4)
    )) %>%

    # drop any entries not in main academic years
    filter(year_group %in% main_year_groups_factor) %>%
    mutate(year_group = factor(year_group,  
                                    ordered = TRUE,  
                                    levels = main_year_groups_factor)) %>%
    rename(academic_year = AcademicYear) %>% 
    arrange(person_id, academic_year) 


### prep exclusions data ###

exclusion_sql_query <- bq_dataset_query(
    query = "SELECT * FROM exclusions_cleaned
             WHERE CAST(SUBSTR(AcademicYear, 1,4) AS INT64) > 2004 
                AND CAST(SUBSTR(AcademicYear, 1,4) AS INT64) < 2019",
    x = "yhcr-prd-phm-bia-core.CB_FDM_DepartmentForEducation"
)

exclusion_data <- bq_table_download(exclusion_sql_query) %>% 
    # aggregate exclusions by person/year - get first exclusion date and n_exclusions
    group_by(person_id, AcademicYear) %>% 
    summarise(first_excl_acad_yr = min(StartDate), 
              n_exclusions = n()) %>% 
    rename(academic_year = AcademicYear) %>%
    ungroup()


## prep demographics data

demo_sql_query <- bq_dataset_query(
    query = "SELECT person_id,  
                CASE 
                    WHEN remapped_gender = 45766034 THEN 'Male' 
                    WHEN remapped_gender = 45766035 THEN 'Female' 
                ELSE NULL
            END AS sex, 
                CASE
                    WHEN REGEXP_EXTRACT(census_ethnicity, r'^(.+?):') IS NOT NULL THEN REGEXP_EXTRACT(census_ethnicity, r'^(.+?):')
                    ELSE NULL
                END AS ethnic_group, 
            FROM `yhcr-prd-phm-bia-core.CY_STAGING_DATABASE.src_DemoGraphics_MASTER` demo
            WHERE EXISTS(
                SELECT person_id 
                FROM `yhcr-prd-phm-bia-core.CY_FDM_DepartmentForEducation.src_census` census 
                WHERE census.person_id = demo.person_id)",
    x = "yhcr-prd-phm-bia-core.CY_FDM_DepartmentForEducation"
)

demo_data <- bq_table_download(demo_sql_query) %>%
    mutate(ethnicity = as.factor(case_when(
               ethnic_group == "White" ~ "White",  
               ethnic_group == "Asian or Asian British" ~ "Asian",  
               !is.na(ethnic_group) ~ "Other")),
           male = sex == "Male") %>%
    select(person_id, ethnicity, male)


### prep joined data ###

# custom function to calculate mins if input is all NA
min_wo_na <- function(x) {
    if (all(is.na(x))) NA else min(x, na.rm=TRUE)
}
                          
asd_sql_query <- bq_dataset_query(
    query = "SELECT person_id, diagnosis_date FROM ASD_master_tab",
    x = "yhcr-prd-phm-bia-core.CY_ASD_data"
)

asd_data = bq_table_download(asd_sql_query) 
                          
asd_exclusions <- census_data %>% 
    left_join(asd_data) %>%
    left_join(exclusion_data) %>% 
    left_join(demo_data) %>% 
    mutate(
        # create ac year start as datetime feature
        year_start = str_sub(academic_year, end=4), 
        year_month_day_start = str_c(year_start, "-09-01"),
        year_start_date = as.Date(year_month_day_start),
        # ---------
        no_diag = is.na(diagnosis_date),
        days_from_to_diag = difftime(year_start_date, diagnosis_date, units="days"),
        one_yr_diag = days_from_to_diag > 365,
        pending_diag = days_from_to_diag <= 365,
        secondary_school = year_group >= 7,
        excluded = !is.na(n_exclusions),
        n_exclusions = ifelse(is.na(n_exclusions), 0, n_exclusions),
        year_group_int = as.integer(year_group) - 1
    ) %>%
    select(-c("year_start", "year_month_day_start")) %>%

    # add "has been excluded before this ac year" feature
    group_by(person_id) %>% 
    mutate(first_excl_ever = min_wo_na(first_excl_acad_yr), 
           has_prev_excl = case_when(
               year_start_date >= first_excl_ever ~ TRUE,  
               year_start_date < first_excl_ever ~ FALSE,  
               is.na(first_excl_ever) ~ FALSE 
           ),
           diag_status = as.factor(case_when(
               one_yr_diag ~ "one_yr_diag",  
               pending_diag ~ "pending_diag",  
               no_diag ~ "no_diag"
           ))) %>%
    ungroup() 
    # -----------
                          

In [ ]:
options(repr.plot.width = 7.5, repr.plot.height = 5, repr.plot.res = 200)
excl_preds <- 
    asd_exclusions %>%    
        mutate(Diagnosis = case_when(   
            no_diag ~ "None",   
            one_yr_diag ~ "Diagnosed",  
            !one_yr_diag & !no_diag ~ "Pending"
        )) %>% 
        mutate(Diagnosis = factor(
            Diagnosis,    
            levels=c('Diagnosed', 'Pending', 'None')
        )) %>%   
        group_by(year_group, Diagnosis) %>%   
        summarise(n = n()) %>% 
        mutate(pct = n / sum(n)) %>% 
        filter(!Diagnosis == "None") %>% 
        mutate(total_diag = sum(n)) %>%   
    ggplot() +
    geom_bar(aes(x=year_group, y=pct), stat="identity", alpha=0.3) +
    geom_line(aes(x=year_group, y=pct, color=Diagnosis, group=Diagnosis) ) + 
    geom_point(aes(x=year_group, y=pct, color=Diagnosis, group=Diagnosis) ) +
    labs(x="Year Group", y="Count")  + 
    theme_classic() + 
    scale_color_manual(values=c("#E69F00", "#56B4E9", "#696969"))
ggsave("plots/excl_pcts.jpg", excl_preds, width=7.5, height=5)
excl_preds

In [ ]:
asd_exclusions <- asd_exclusions %>%
    # keep only individuals with 10 or more academic years in data
    group_by(person_id) %>% 
    mutate(person_entries = n()) %>% 
    ungroup() %>%
    filter(person_entries >= 10)

In [ ]:
options(repr.plot.width = 7.5, repr.plot.height = 5, repr.plot.res = 200)
excl_preds <- 
    asd_exclusions %>%    
        mutate(Diagnosis = case_when(   
            no_diag ~ "None",   
            one_yr_diag ~ "Diagnosed",  
            !one_yr_diag & !no_diag ~ "Pending"
        )) %>% 
        mutate(Diagnosis = factor(
            Diagnosis,    
            levels=c('Diagnosed', 'Pending', 'None')
        )) %>%   
        group_by(year_group, Diagnosis) %>%   
        summarise(n = n()) %>% 
        mutate(pct = n / sum(n)) %>% 
        filter(!Diagnosis == "None") %>% 
        mutate(total_diag = sum(n)) %>%   
    ggplot() +
    geom_bar(aes(x=year_group, y=pct), stat="identity", alpha=0.3) +
    geom_line(aes(x=year_group, y=pct, color=Diagnosis, group=Diagnosis) ) + 
    geom_point(aes(x=year_group, y=pct, color=Diagnosis, group=Diagnosis) ) +
    labs(x="Year Group", y="Count")  + 
    theme_classic() + 
    scale_color_manual(values=c("#E69F00", "#56B4E9", "#696969"))
ggsave("plots/excl_pcts.jpg", excl_preds, width=7.5, height=5)
excl_preds

In [ ]:
options(repr.plot.width = 7.5, repr.plot.height = 5, repr.plot.res = 200)
excl_preds <- ggplot(
    asd_exclusions %>%   
    mutate(Diagnosis = case_when(  
        no_diag ~ "None",  
        one_yr_diag ~ "Diagnosed", 
        !one_yr_diag & !no_diag ~ "Pending")) %>%
    mutate(Diagnosis = factor(Diagnosis,  
                              levels=c('Diagnosed', 'Pending', 'None'))) %>%  
    group_by(year_group, Diagnosis) %>%  
    summarise(n = n(),    
              n_excluded = sum(excluded),    
              pct_excluded = (n_excluded / n)*100),   
    aes(x=year_group,    
        y=pct_excluded,   
        color=Diagnosis,  
        group=Diagnosis)) + 
    geom_line() + 
    geom_point() +
    labs(x="Year Group", y="Percentage Excluded")  + 
    theme_classic() + 
    scale_color_manual(values=c("#E69F00", "#56B4E9", "#696969"))
ggsave("plots/excl_pcts.jpg", excl_preds, width=7.5, height=5)
excl_preds

In [ ]:
full_simple_model <- glm(excluded ~ diag_status + has_prev_excl + secondary_school + year_group_int + male + ethnicity,
             family = binomial(link="logit"),
             data = asd_exclusions)
full_simple_model$name <- "full_simple_model"

In [ ]:
summary(full_simple_model)

In [ ]:
exp(cbind("Odds ratio" = coef(full_simple_model), confint.default(full_simple_model, level = 0.95)))

In [ ]:
length(full_simple_model$residuals)

In [ ]:
nrow(asd_exclusions)

In [ ]:
resids <- asd_exclusions %>%  
    select(diag_status, has_prev_excl, secondary_school, year_group, male, ethnicity) %>% 
    drop_na() %>%
    select(year_group)
resids$residual <- full_simple_model$residuals

In [ ]:
resids %>%
    ggplot(aes(x=year_group, y=log(residual))) +
    geom_boxplot()